In [1]:
import os

import optuna
import pandas as pd
import numpy as np
import torch
import copy

from settings.config import *
from src.training.commons import init_optuna_storage
from src.commons.exp_config import HTunerExpConfig, HGeneratorConfig, ExpConfig
from src.models.custom_schedulers import ConstantStartReduceOnPlateau
from src.data_processing.transformations import transform_aug_adv
from src.models.dummy import DummyBNModel
from src.models.resnet import ResnetLikeV1, Resnet18

In [3]:
exp_dir_h = os.path.join(EXPERIMENTS_PATH, "basic")
exp_name_h = "resnets_htuning{25k}"

exp_dir_f = os.path.join(EXPERIMENTS_PATH, "basic")
exp_name_f = "resnets_training_BM_F1_INGS"


In [4]:
htuning_config = HTunerExpConfig.load_from_file(str(os.path.join(exp_dir_h, exp_name_h, HTUNER_CONFIG_FILE)))
htuning_config.hp

{'lgn_model_type': src.lightning.lgn_models.BaseWithSchedulerLGNM,
 'batch_size': 128,
 'lr': 0.001,
 'loss_fn': torch.nn.modules.loss.BCEWithLogitsLoss,
 'weighted_loss': True,
 'optimizer': torch.optim.adam.Adam,
 'momentum': None,
 'weight_decay': None,
 'use_swa': False,
 'lr_scheduler': None,
 'lr_scheduler_params': {},
 'torch_model': {'type': src.models.dummy.DummyModel,
  'input_shape': [224, 224],
  'num_classes': 183,
  'trns_aug': None,
  'trns_bld_aug': None,
  'trns_bld_plain': None,
  'lp_phase': None},
 'metrics': {'acc': {'type': torchmetrics.classification.accuracy.Accuracy,
   'init_params': {'task': 'multilabel',
    'num_labels': None,
    'average': 'weighted'},
   'logging_params': {'prog_bar': False, 'on_epoch': True, 'on_step': False}},
  'precision': {'type': torchmetrics.classification.precision_recall.Precision,
   'init_params': {'task': 'multilabel',
    'num_labels': None,
    'average': 'weighted'},
   'logging_params': {'prog_bar': False, 'on_epoch': Tru

In [12]:
TRIAL_NAME_PREFIX = "trial_"

In [13]:
trial_configs = []
for trial_id in range(0, 4):
    checkpoint_data = torch.load(str(os.path.join(exp_dir_f, exp_name_f, f"{TRIAL_NAME_PREFIX}{trial_id}", "best_model.ckpt")), weights_only=False)
    trial_config = ExpConfig.load_from_ckpt_data(checkpoint_data)
    trial_configs.append(trial_config)

In [14]:
trial_configs[0].datamodule

{'type': src.data_processing.images_recipes.ImagesRecipesBaseDataModule,
 'image_shape': (224, 224),
 'data_dir': 'H:\\programmazione\\python_projects\\ingredient_recognition\\data\\input\\yummly',
 'metadata_filename': 'metadata.json',
 'category': 'all',
 'feature_label': 'ingredients_ok',
 'num_workers': 8,
 'label_encoder': {'type': src.data_processing.labels_encoders.MultiLabelBinarizerRobust,
  'classes': ['allspice',
   'almond',
   'ancho chiles',
   'anchovy fillets',
   'apple',
   'apricots',
   'artichoke',
   'arugula',
   'asparagus',
   'avocado',
   'baby arugula',
   'bacon',
   'baguette',
   'baking powder',
   'banana',
   'basil',
   'bay leaf',
   'bay leaves',
   'beans',
   'beef',
   'beer',
   'beet',
   'black olives',
   'blackberries',
   'blueberries',
   'bread',
   'broccoli',
   'brussels sprouts',
   'bunch cilantro',
   'butter',
   'cabbage',
   'capers',
   'cardamom',
   'carrot',
   'cashew',
   'cauliflower',
   'cayenne',
   'celery',
   'celery

In [18]:
htuning_config.datamodule == trial_configs[0].datamodule

False

In [6]:

trial_df = pd.read_csv(str(os.path.join(exp_dir, exp_name, "trials_info.csv")), index_col=0)


In [7]:
trial_df

,best_epoch,lr,lr_scheduler,optimizer,model_type,image_augmentation,warm_duration,weight_decay,weighted_loss,val_acc,val_hamming,val_loss,val_precision,val_recall,train_acc,train_hamming,train_loss,train_precision,train_recall
trial,,,,,,,,,,,,,,,,,,,
0,39,0.001507,CosineAnnealingWarmRestarts,sgd,resnet18Pre,True,NaN,0.007222,True,0.794852,0.205148,0.158086,0.171629,0.159537,0.794136,0.205864,0.154997,0.170031,0.160831
1,1,0.003736,ConstantStartReduceOnPlateau,adam,resnet18Like,False,15.0,0.001839,False,0.784275,0.215726,0.146443,0.101094,0.157141,0.790094,0.209906,0.136003,0.136531,0.155008
2,1,0.001122,CosineAnnealingWarmRestarts,adam,resnet18Like,True,NaN,0.000863,True,0.798348,0.201652,0.141371,0.171878,0.150372,0.797928,0.202072,0.140182,0.181304,0.188411
3,19,0.003883,CosineAnnealingWarmRestarts,adam,dummy,False,NaN,0.006753,True,0.797489,0.202511,0.142973,0.157874,0.180566,0.796360,0.203640,0.140180,0.166144,0.157544
4,33,0.075384,ConstantStartReduceOnPlateau,adam,resnet18Like,False,10.0,0.004200,True,0.787946,0.212055,0.164222,0.101116,0.181041,0.786269,0.213731,0.163350,0.082166,0.143569
5,3,0.038293,none,sgd,resnet18Like,False,NaN,0.005315,True,0.797742,0.202258,0.147268,0.157994,0.195159,0.797482,0.202518,0.147326,0.162942,0.177444
6,25,0.004479,ConstantStartReduceOnPlateau,adam,dummy,False,0.0,0.003421,True,0.800791,0.199209,0.141827,0.168603,0.176741,0.797930,0.202070,0.137772,0.165274,0.164444
7,9,0.002269,CosineAnnealingWarmRestarts,adam,dummy,False,NaN,0.008744,False,0.795412,0.204588,0.142825,0.161691,0.146151,0.796746,0.203255,0.144790,0.162912,0.169819
8,35,0.006033,ConstantStartReduceOnPlateau,adam,resnet18Pre,True,0.0,0.000822,True,0.801241,0.198759,0.143620,0.160473,0.209702,0.800599,0.199401,0.139582,0.163860,0.185785


In [8]:
trial_configs = []
for trial_id, trial_info in trial_df.iterrows():
    trial_dir = os.path.join(exp_dir, exp_name, f"{TRIAL_NAME_PREFIX}{trial_id}")

    trial_config = copy.deepcopy(general_config)
    trial_config.update_config(
        hp_lr=trial_info["lr"],
        hp_lr_scheduler=None,
        hp_optimizer=(torch.optim.Adam if trial_info["optimizer"] == "adam" else torch.optim.SGD),
        tm_type=(DummyBNModel if trial_info['model_type'] == "dummy" else (ResnetLikeV1 if trial_info['model_type'] == "resnet18Like" else Resnet18)),
        hp_weighted_loss=trial_info['weighted_loss'],
        hp_weight_decay=trial_info['weight_decay'],
        hp_momentum=0.9,
        tm_trns_aug=(transform_aug_adv if trial_info['image_augmentation'] else None),
    )
    if trial_info['lr_scheduler'] == "ConstantStartReduceOnPlateau":
        trial_config.update_config(hp_lr_scheduler=ConstantStartReduceOnPlateau,
                                      hp_lr_scheduler_params={"warm_duration": trial_info['warm_duration'], "mode": "min", "factor": 0.05, "patience": 5, "cooldown":1, "min_lr": DEF_LR/1e3})
    elif trial_info['lr_scheduler'] == "CosineAnnealingWarmRestarts":
        trial_config.update_config(hp_lr_scheduler=torch.optim.lr_scheduler.CosineAnnealingWarmRestarts, hp_lr_scheduler_params={"T_0": 5, "T_mult": 2, "eta_min": DEF_LR/1e3})
    
    trial_config.save_to_file(os.path.join(trial_dir, HTUNING_TRIAL_CONFIG_FILE))
    # models_chkpt.append(str(os.path.join(exp_dir, exp_name, f"trial_{trial_id}", "best_model.ckpt")))
    trial_configs.append(trial_config)


In [9]:
trial_configs[0].trainer

{'type': src.lightning.lgn_trainers.OptunaTrainer,
 'debug': False,
 'max_epochs': 40,
 'save_dir': 'H:\\programmazione\\python_projects\\ingredient_recognition\\experiments\\basic\\resnets_htuning{25k}',
 'limit_train_batches': 0.5,
 'limit_predict_batches': 1,
 'log_every_n_steps': 50,
 'early_stop': None}

In [9]:
trial_config = ExpConfig.load_from_file(str(os.path.join(exp_dir, exp_name, f"{TRIAL_NAME_PREFIX}{best_trials.index[0]}", HTUNING_TRIAL_CONFIG_FILE)))
trial_config.hp

{'type': src.lightning.lgn_trainers.OptunaTrainer,
 'debug': False,
 'max_epochs': 40,
 'save_dir': 'H:\\programmazione\\python_projects\\ingredient_recognition\\experiments\\basic\\resnets_htuning{25k}',
 'limit_train_batches': 0.5,
 'limit_predict_batches': 1,
 'log_every_n_steps': 50,
 'early_stop': None}

In [6]:
trial_id = 0
exp_config = trial_configs[trial_id]

model_config, trainer_config = exp_config.lgn_model, exp_config.trainer

lgn_model = model_config['lgn_model_type'].load_from_config(model_config)
lgn_model.load_weights_from_checkpoint(models_chkpt[trial_id])

dm_config = exp_config.datamodule
data_module = dm_config['type'].load_from_config(dm_config, batch_size=lgn_model.batch_size, transform_aug=lgn_model.transform_aug, transform_plain=lgn_model.transform_plain)



In [36]:
lgn_model.

[Resize(size=[224, 224], interpolation=InterpolationMode.BILINEAR, antialias=True),
 RandomResizedCrop(size=(224, 224), scale=(0.33, 1.0), ratio=(0.5, 2.0), interpolation=InterpolationMode.BILINEAR, antialias=True),
 RandomHorizontalFlip(p=0.3),
 RandomVerticalFlip(p=0.3),
 RandomRotation(degrees=[-0.2, 0.2], interpolation=InterpolationMode.NEAREST, expand=False, fill=0)]

{'hyper_parameters': {'lgn_model_type': src.lightning.lgn_models.BaseLGNM,
  'batch_size': 128,
  'lr': 0.0439108278045022,
  'loss_fn': torch.nn.modules.loss.BCEWithLogitsLoss,
  'weighted_loss': True,
  'optimizer': torch.optim.sgd.SGD,
  'momentum': 0.9,
  'weight_decay': 0.000212602939019,
  'use_swa': False,
  'lr_scheduler': src.models.custom_schedulers.ConstantStartReduceOnPlateau,
  'lr_scheduler_params': {'warm_duration': 0.0,
   'mode': 'min',
   'factor': 0.05,
   'patience': 5,
   'cooldown': 1,
   'min_lr': 1e-06},
  'torch_model': {'type': src.models.resnet.ResnetLikeV1,
   'input_shape': [224, 224],
   'num_classes': 183,
   'trns_aug': <function src.data_processing.transformations.transform_aug_adv(image_shape: Tuple[int, int] = (224, 224)) -> List[torchvision.transforms.v2._transform.Transform]>,
   'trns_bld_aug': None,
   'trns_bld_plain': None,
   'lp_phase': None},
  'metrics': {'acc': {'type': torchmetrics.classification.accuracy.Accuracy,
    'init_params': {'tas